# Email Spam Detection using BERT

This notebook implements a BERT-based email spam detection system using the DistilBERT model. The system will:
1. Load and preprocess the email dataset
2. Tokenize the data using BERT tokenizer
3. Fine-tune a pre-trained DistilBERT model
4. Evaluate the model performance
5. Generate a comprehensive report with visualizations

## 1. Import Required Libraries

In [2]:
# Standard libraries
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_curve, auc,
                             precision_recall_curve, average_precision_score, matthews_corrcoef)

# PyTorch and Transformers
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Plotly for visualizations
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")

d:\Spam-and-Malware-Detection-AI-model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version: 2.9.0+cpu
CUDA Available: False


## 2. Configuration and Device Setup

In [3]:
# Set random seeds for reproducibility
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Training configuration
NUM_EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
MAX_LENGTH = 128  # Maximum sequence length for BERT

print(f"\nTraining Configuration:")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Max Sequence Length: {MAX_LENGTH}")

Using device: cpu

Training Configuration:
  Epochs: 1
  Batch Size: 16
  Learning Rate: 5e-05
  Max Sequence Length: 128


## 3. Load and Explore the Email Dataset

In [4]:
# Load the email dataset
df = pd.read_csv('Dataset/raw_email_data.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nDataset Info:")
print(df.info())
print(f"\nFirst few rows:")
df.head()

Dataset Shape: (5572, 2)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None

First few rows:


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Explore the distribution of spam vs ham
print("Class Distribution:")
print(df['Category'].value_counts())
print(f"\nClass Distribution (%):")
print(df['Category'].value_counts(normalize=True) * 100)

# Visualize the distribution using Plotly
from plotly.subplots import make_subplots

class_counts = df['Category'].value_counts()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Email Category Distribution', 'Email Category Distribution (%)'),
    specs=[[{'type':'bar'}, {'type':'pie'}]]
)

# Bar chart
fig.add_trace(
    go.Bar(x=class_counts.index, y=class_counts.values,
           marker=dict(color=['#2ecc71', '#e74c3c']),
           text=class_counts.values,
           textposition='auto',
           name='Count'),
    row=1, col=1
)

# Pie chart
fig.add_trace(
    go.Pie(labels=class_counts.index, values=class_counts.values,
           marker=dict(colors=['#2ecc71', '#e74c3c']),
           textinfo='label+percent',
           name='Distribution'),
    row=1, col=2
)

fig.update_layout(
    title_text="Email Dataset Class Distribution",
    showlegend=False,
    height=500,
    width=1200
)

fig.update_xaxes(title_text="Category", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()

Class Distribution:
Category
ham     4825
spam     747
Name: count, dtype: int64

Class Distribution (%):
Category
ham     86.593683
spam    13.406317
Name: proportion, dtype: float64


In [6]:
# Analyze message lengths
df['message_length'] = df['Message'].apply(len)
df['word_count'] = df['Message'].apply(lambda x: len(str(x).split()))

print("Message Length Statistics:")
print(df.groupby('Category')[['message_length', 'word_count']].describe())

# Visualize message lengths using Plotly
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Message Length Distribution by Category', 
                    'Word Count Distribution by Category')
)

categories = df['Category'].unique()
colors_map = {'ham': '#2ecc71', 'spam': '#e74c3c'}

for category in categories:
    data = df[df['Category'] == category]['message_length']
    fig.add_trace(
        go.Box(y=data, name=category.capitalize(), 
               marker_color=colors_map[category],
               boxmean='sd'),
        row=1, col=1
    )
    
    data = df[df['Category'] == category]['word_count']
    fig.add_trace(
        go.Box(y=data, name=category.capitalize(),
               marker_color=colors_map[category],
               boxmean='sd',
               showlegend=False),
        row=1, col=2
    )

fig.update_yaxes(title_text="Message Length (characters)", row=1, col=1)
fig.update_yaxes(title_text="Word Count", row=1, col=2)

fig.update_layout(
    title_text="Message Length and Word Count Analysis",
    height=500,
    width=1200,
    showlegend=True
)

fig.show()

Message Length Statistics:
         message_length                                                  \
                  count       mean        std  min    25%    50%    75%   
Category                                                                  
ham              4825.0   71.44829  58.434864  2.0   33.0   52.0   93.0   
spam              747.0  137.98929  29.980287  7.0  132.0  149.0  157.0   

                word_count                                                      
            max      count       mean        std  min   25%   50%   75%    max  
Category                                                                        
ham       910.0     4825.0  14.310259  11.517945  1.0   7.0  11.0  19.0  171.0  
spam      223.0      747.0  23.812584   5.956923  1.0  22.0  25.0  28.0   35.0  


## 4. Data Preprocessing

In [7]:
# Convert labels to binary (ham=0, spam=1)
label_mapping = {'ham': 0, 'spam': 1}
df['label'] = df['Category'].map(label_mapping)

print(f"Label Mapping: {label_mapping}")
print(f"\nLabel Distribution:")
print(df['label'].value_counts())

# Check for missing values
print(f"\nMissing Values:")
print(df.isnull().sum())

Label Mapping: {'ham': 0, 'spam': 1}

Label Distribution:
label
0    4825
1     747
Name: count, dtype: int64

Missing Values:
Category          0
Message           0
message_length    0
word_count        0
label             0
dtype: int64


In [8]:
# Split the dataset into train, validation, and test sets
# First split: 80% train+val, 20% test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
    df['Message'].values, 
    df['label'].values, 
    test_size=0.2, 
    random_state=RANDOM_SEED,
    stratify=df['label'].values
)

# Second split: 75% train, 25% validation (of the 80%)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_val_texts, 
    train_val_labels, 
    test_size=0.25, 
    random_state=RANDOM_SEED,
    stratify=train_val_labels
)

print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")

print(f"\nTraining set spam ratio: {train_labels.sum() / len(train_labels) * 100:.2f}%")
print(f"Validation set spam ratio: {val_labels.sum() / len(val_labels) * 100:.2f}%")
print(f"Test set spam ratio: {test_labels.sum() / len(test_labels) * 100:.2f}%")

Training set size: 3342
Validation set size: 1115
Test set size: 1115

Training set spam ratio: 13.41%
Validation set spam ratio: 13.45%
Test set spam ratio: 13.36%


## 5. Tokenization with BERT Tokenizer

In [9]:
# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

print("Tokenizer loaded successfully!")
print(f"Vocabulary size: {tokenizer.vocab_size}")

# Example tokenization
sample_text = train_texts[0]
print(f"\nSample Text: {sample_text[:100]}...")
print(f"\nTokenized: {tokenizer.tokenize(sample_text)[:20]}...")

Tokenizer loaded successfully!
Vocabulary size: 30522

Sample Text: Am also doing in cbe only. But have to pay....

Tokenized: ['am', 'also', 'doing', 'in', 'cbe', 'only', '.', 'but', 'have', 'to', 'pay', '.']...


In [10]:
# Tokenize all datasets
print("Tokenizing datasets...")
train_encodings = tokenizer(
    list(train_texts), 
    truncation=True, 
    padding=True, 
    max_length=MAX_LENGTH
)
val_encodings = tokenizer(
    list(val_texts), 
    truncation=True, 
    padding=True, 
    max_length=MAX_LENGTH
)
test_encodings = tokenizer(
    list(test_texts), 
    truncation=True, 
    padding=True, 
    max_length=MAX_LENGTH
)

print("Tokenization complete!")
print(f"Training encoding keys: {train_encodings.keys()}")
print(f"First training sample input_ids shape: {len(train_encodings['input_ids'][0])}")

Tokenizing datasets...
Tokenization complete!
Training encoding keys: KeysView({'input_ids': [[101, 2572, 2036, 2725, 1999, 15852, 2069, 1012, 2021, 2031, 2000, 3477, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2008, 2052, 2022, 2204, 1529, 1045, 1005, 2222, 3042, 2017, 3419, 2080, 6265, 7292, 1010, 4618, 1045, 1010, 2000, 22933, 2242, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1998, 2129, 2017

## 6. Create PyTorch Datasets and DataLoaders

In [11]:
# Create custom Dataset class
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = EmailDataset(train_encodings, train_labels)
val_dataset = EmailDataset(val_encodings, val_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 3342
Validation dataset size: 1115
Test dataset size: 1115


In [12]:
# Create DataLoaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")
print(f"Number of test batches: {len(test_loader)}")

Number of training batches: 209
Number of validation batches: 70
Number of test batches: 70


## 7. Load Pre-trained DistilBERT Model

In [13]:
# Load pre-trained DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2  # Binary classification: spam or ham
)

# Move model to device
model.to(DEVICE)

print("Model loaded successfully!")
print(f"Model device: {next(model.parameters()).device}")
print(f"\nModel architecture:")
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!
Model device: cpu

Model architecture:
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
         

## 8. Define Training Functions

In [14]:
# Function to compute accuracy
def compute_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            # Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, dim=1)
            
            # Calculate accuracy
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
    
    return correct_pred.float() / num_examples * 100

print("Accuracy computation function defined!")

Accuracy computation function defined!


In [15]:
# Function to get predictions and true labels
def get_predictions(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            
            # Get probabilities using softmax
            probabilities = F.softmax(logits, dim=1)
            predicted_labels = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predicted_labels.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_labels), np.array(all_probabilities)

print("Prediction function defined!")

Prediction function defined!


## 9. Train the Model

In [16]:
# Set up optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_acc': [],
    'epoch_times': []
}

print("Starting training...")
print("=" * 70)

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start_time = time.time()
    model.train()
    
    epoch_loss = 0
    batch_losses = []
    
    for batch_idx, batch in enumerate(train_loader):
        # Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track loss
        batch_losses.append(loss.item())
        epoch_loss += loss.item()
        
        # Logging - Print at start and every 50 batches
        if batch_idx % 50 == 0:
            print(f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                  f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                  f'Loss: {loss.item():.4f}')
    
    # Calculate metrics
    avg_loss = epoch_loss / len(train_loader)
    model.eval()
    
    with torch.set_grad_enabled(False):
        train_acc = compute_accuracy(model, train_loader, DEVICE)
        val_acc = compute_accuracy(model, val_loader, DEVICE)
    
    epoch_time = time.time() - epoch_start_time
    
    # Store history
    history['train_loss'].append(avg_loss)
    history['train_acc'].append(train_acc.item())
    history['val_acc'].append(val_acc.item())
    history['epoch_times'].append(epoch_time)
    
    print(f'\n{"="*70}')
    print(f'Epoch {epoch+1}/{NUM_EPOCHS} Summary:')
    print(f'  Average Loss: {avg_loss:.4f}')
    print(f'  Training Accuracy: {train_acc:.2f}%')
    print(f'  Validation Accuracy: {val_acc:.2f}%')
    print(f'  Epoch Time: {epoch_time:.2f}s')
    print(f'{"="*70}\n')

total_training_time = time.time() - start_time
print(f'\nTotal Training Time: {total_training_time/60:.2f} minutes')
print("Training completed!")

Starting training...
Epoch: 0001/0001 | Batch 0000/0209 | Loss: 0.7106
Epoch: 0001/0001 | Batch 0000/0209 | Loss: 0.7106
Epoch: 0001/0001 | Batch 0050/0209 | Loss: 0.2314
Epoch: 0001/0001 | Batch 0050/0209 | Loss: 0.2314
Epoch: 0001/0001 | Batch 0100/0209 | Loss: 0.0125
Epoch: 0001/0001 | Batch 0100/0209 | Loss: 0.0125
Epoch: 0001/0001 | Batch 0150/0209 | Loss: 0.0032
Epoch: 0001/0001 | Batch 0150/0209 | Loss: 0.0032
Epoch: 0001/0001 | Batch 0200/0209 | Loss: 0.0253

Epoch 1/1 Summary:
  Average Loss: 0.0754
  Training Accuracy: 99.37%
  Validation Accuracy: 98.74%
  Epoch Time: 908.53s


Total Training Time: 15.14 minutes
Training completed!


## 10. Evaluate on Test Set

In [25]:
# Evaluate on test set
print("Evaluating on test set...")
test_acc = compute_accuracy(model, test_loader, DEVICE)
print(f'Test Accuracy: {test_acc:.2f}%')

# Get predictions
test_predictions, test_true_labels, test_probabilities = get_predictions(
    model, test_loader, DEVICE
)

print(f"\nTest set predictions obtained!")
print(f"Total test samples: {len(test_predictions)}")

Evaluating on test set...
Test Accuracy: 99.19%

Test set predictions obtained!
Total test samples: 1115


## 11. Calculate Detailed Metrics

In [26]:
# Calculate comprehensive metrics
accuracy = accuracy_score(test_true_labels, test_predictions)
precision = precision_score(test_true_labels, test_predictions)
recall = recall_score(test_true_labels, test_predictions)
f1 = f1_score(test_true_labels, test_predictions)

# Confusion matrix components
cm = confusion_matrix(test_true_labels, test_predictions)
tn, fp, fn, tp = cm.ravel()

# Additional metrics
tnr = tn / (tn + fp)  # True Negative Rate / Specificity
fpr = fp / (fp + tn)  # False Positive Rate
fnr = fn / (fn + tp)  # False Negative Rate
npv = tn / (tn + fn)  # Negative Predictive Value
mcc = matthews_corrcoef(test_true_labels, test_predictions)  # Matthews Correlation Coefficient

# Precision-Recall Curve and AUC
precision_curve, recall_curve, _ = precision_recall_curve(test_true_labels, test_probabilities[:, 1])
pr_auc = average_precision_score(test_true_labels, test_probabilities[:, 1])




## 12. Confusion Matrix Visualization

In [27]:
# Confusion Matrix Visualization using Plotly
cm = confusion_matrix(test_true_labels, test_predictions)

# Create subplots for count and percentage
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Confusion Matrix (Counts)', 'Confusion Matrix (Percentages)'),
    specs=[[{'type':'heatmap'}, {'type':'heatmap'}]]
)

# Confusion matrix with counts
fig.add_trace(
    go.Heatmap(
        z=cm,
        x=['Ham', 'Spam'],
        y=['Ham', 'Spam'],
        text=cm,
        texttemplate='%{text}',
        textfont={"size": 20},
        colorscale='Blues',
        showscale=True,
        colorbar=dict(x=0.46)
    ),
    row=1, col=1
)

# Confusion matrix with percentages
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
fig.add_trace(
    go.Heatmap(
        z=cm_percent,
        x=['Ham', 'Spam'],
        y=['Ham', 'Spam'],
        text=[[f'{val:.2f}%' for val in row] for row in cm_percent],
        texttemplate='%{text}',
        textfont={"size": 20},
        colorscale='Greens',
        showscale=True,
        colorbar=dict(x=1.0)
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Predicted Label", row=1, col=1)
fig.update_xaxes(title_text="Predicted Label", row=1, col=2)
fig.update_yaxes(title_text="True Label", row=1, col=1)
fig.update_yaxes(title_text="True Label", row=1, col=2)

fig.update_layout(
    title_text="Confusion Matrix Visualization",
    height=500,
    width=1200
)

fig.show()


## 13. Generate Comprehensive Report

In [ ]:
# Generate comprehensive report
report = f"""
{'='*110}
                        EMAIL SPAM DETECTION - COMPREHENSIVE REPORT
                              Using DistilBERT Transformer Model
{'='*110}

1. DATASET INFORMATION
{'-'*110}
   Total Samples: {len(df)}
   Training Samples: {len(train_texts)} ({len(train_texts)/len(df)*100:.1f}%)
   Validation Samples: {len(val_texts)} ({len(val_texts)/len(df)*100:.1f}%)
   Test Samples: {len(test_texts)} ({len(test_texts)/len(df)*100:.1f}%)
   
   Class Distribution (Original):
   - Ham emails: {(df['label'] == 0).sum()} ({(df['label'] == 0).sum()/len(df)*100:.1f}%)
   - Spam emails: {(df['label'] == 1).sum()} ({(df['label'] == 1).sum()/len(df)*100:.1f}%)

2. MODEL CONFIGURATION
{'-'*110}
   Model: DistilBERT (distilbert-base-uncased)
   Max Sequence Length: {MAX_LENGTH}
   Batch Size: {BATCH_SIZE}
   Learning Rate: {LEARNING_RATE}
   Number of Epochs: {NUM_EPOCHS}
   Optimizer: Adam
   Device: {DEVICE}

3. TRAINING PERFORMANCE
{'-'*110}
   Total Training Time: {total_training_time/60:.2f} minutes
   Average Time per Epoch: {np.mean(history['epoch_times']):.2f} seconds
   
   Final Training Metrics:
   - Training Accuracy: {history['train_acc'][-1]:.2f}%
   - Validation Accuracy: {history['val_acc'][-1]:.2f}%
   - Training Loss: {history['train_loss'][-1]:.4f}
   
   Convergence Analysis:
   - Loss Reduction: {((history['train_loss'][0] - history['train_loss'][-1])/history['train_loss'][0]*100):.2f}%
   - Generalization Gap: {(history['train_acc'][-1] - history['val_acc'][-1]):.2f}%

4. COMPREHENSIVE TEST SET METRICS (Ordered as Requested)
{'-'*110}
   1. Area Under Precision-Recall Curve: {pr_auc:.4f}
   2. TPR (True Positive Rate / Recall / Sensitivity): {recall:.4f}
   3. Precision: {precision:.4f}
   4. Positive F1-Score: {f1:.4f}
   5. TNR (True Negative Rate / Specificity): {tnr:.4f}
   6. NPV (Negative Predictive Value): {npv:.4f}
   7. FPR (False Positive Rate): {fpr:.4f}
   8. FNR (False Negative Rate): {fnr:.4f}
   9. MCC (Matthews Correlation Coefficient): {mcc:.4f}
   
   Additional Metrics:
   - Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)

5. CONFUSION MATRIX ANALYSIS
{'-'*110}
   True Negatives (Ham → Ham): {cm[0, 0]} ({cm[0, 0]/cm.sum()*100:.2f}%)
   False Positives (Ham → Spam): {cm[0, 1]} ({cm[0, 1]/cm.sum()*100:.2f}%)
   False Negatives (Spam → Ham): {cm[1, 0]} ({cm[1, 0]/cm.sum()*100:.2f}%)
   True Positives (Spam → Spam): {cm[1, 1]} ({cm[1, 1]/cm.sum()*100:.2f}%)
   
   Error Analysis:
   - Total Errors: {cm[0, 1] + cm[1, 0]} ({(cm[0, 1] + cm[1, 0])/cm.sum()*100:.2f}%)
   - Ham Misclassified as Spam: {cm[0, 1]}
   - Spam Misclassified as Ham: {cm[1, 0]}


{'='*110}
                                    END OF REPORT
{'='*110}
"""

print(report)

# Save report to file
with open('spam_detection_bert_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✓ Comprehensive report saved to 'spam_detection_bert_report.txt'")


                        EMAIL SPAM DETECTION - COMPREHENSIVE REPORT
                              Using DistilBERT Transformer Model

1. DATASET INFORMATION
--------------------------------------------------------------------------------------------------------------
   Total Samples: 5572
   Training Samples: 3342 (60.0%)
   Validation Samples: 1115 (20.0%)
   Test Samples: 1115 (20.0%)

   Class Distribution (Original):
   - Ham emails: 4825 (86.6%)
   - Spam emails: 747 (13.4%)

2. MODEL CONFIGURATION
--------------------------------------------------------------------------------------------------------------
   Model: DistilBERT (distilbert-base-uncased)
   Parameters: ~66 million
   Max Sequence Length: 128
   Batch Size: 16
   Learning Rate: 5e-05
   Number of Epochs: 1
   Optimizer: Adam
   Device: cpu

3. TRAINING PERFORMANCE
--------------------------------------------------------------------------------------------------------------
   Total Training Time: 15.14 minutes
   

## 14. Save the Trained Model

In [29]:
# Save the trained model and related components
print("\n" + "="*80)
print("SAVING MODEL AND COMPONENTS")
print("="*80)

import os
import json
import joblib
from pathlib import Path

# Create directory for saved model
models_dir = Path('.') / 'saved_models' / 'bert_spam_detector'
models_dir.mkdir(parents=True, exist_ok=True)

# Save BERT model and tokenizer (using Hugging Face's save methods)
model.save_pretrained(models_dir)
tokenizer.save_pretrained(models_dir)
print(f"✓ BERT model saved to: {models_dir / 'model.safetensors'}")
print(f"✓ Tokenizer saved to: {models_dir / 'tokenizer.json'}")

# Save label mapping (for consistency with XGBoost approach)
label_mapping_path = models_dir / 'label_mapping.joblib'
joblib.dump(label_mapping, label_mapping_path)
print(f"✓ Label mapping saved to: {label_mapping_path}")

# Save comprehensive metadata about the model
metadata = {
    'model_name': 'DistilBERT Email Spam Detector',
    'model_architecture': 'distilbert-base-uncased',
    'num_parameters': '~66 million',
    'task': 'Binary Text Classification (Spam/Ham)',
    
    # Training configuration
    'training_config': {
        'num_epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'max_sequence_length': MAX_LENGTH,
        'optimizer': 'Adam',
        'device': str(DEVICE),
        'random_seed': RANDOM_SEED
    },
    
    # Dataset information
    'dataset_info': {
        'total_samples': len(df),
        'train_samples': len(train_texts),
        'val_samples': len(val_texts),
        'test_samples': len(test_texts),
        'spam_ratio': float((df['label'] == 1).sum() / len(df))
    },
    
    # Training performance
    'training_metrics': {
        'final_train_loss': float(history['train_loss'][-1]),
        'final_train_accuracy': float(history['train_acc'][-1]),
        'final_val_accuracy': float(history['val_acc'][-1]),
        'total_training_time_minutes': float(total_training_time / 60)
    },
    
    # Test set performance (comprehensive metrics in order)
    'test_metrics': {
        'area_under_pr_curve': float(pr_auc),
        'tpr_recall_sensitivity': float(recall),
        'precision': float(precision),
        'f1_score': float(f1),
        'tnr_specificity': float(tnr),
        'npv': float(npv),
        'fpr': float(fpr),
        'fnr': float(fnr),
        'mcc': float(mcc),
        'accuracy': float(accuracy)
    },
    
    # Confusion matrix
    'confusion_matrix': {
        'true_negatives': int(cm[0, 0]),
        'false_positives': int(cm[0, 1]),
        'false_negatives': int(cm[1, 0]),
        'true_positives': int(cm[1, 1])
    },
    
    
    # Training history
    'training_history': {
        'train_loss': [float(x) for x in history['train_loss']],
        'train_acc': [float(x) for x in history['train_acc']],
        'val_acc': [float(x) for x in history['val_acc']],
        'epoch_times': [float(x) for x in history['epoch_times']]
    }
}

# Save metadata as JSON (human-readable)
metadata_json_path = models_dir / 'model_metadata.json'
with open(metadata_json_path, 'w') as f:
    json.dump(metadata, f, indent=4)
print(f"✓ Model metadata saved to: {metadata_json_path}")

# Also save as joblib (for easy Python loading)
metadata_joblib_path = models_dir / 'model_metadata.joblib'
joblib.dump(metadata, metadata_joblib_path)
print(f"✓ Model metadata (joblib) saved to: {metadata_joblib_path}")

print("\n" + "="*80)
print("MODEL COMPONENTS SAVED SUCCESSFULLY")
print("="*80)
print(f"\nSaved files:")
print(f"  1. model.safetensors - Trained BERT model weights")
print(f"  2. config.json - Model configuration")
print(f"  3. tokenizer.json, vocab.txt - Tokenizer files")
print(f"  4. label_mapping.joblib - Label mapping (ham=0, spam=1)")
print(f"  5. model_metadata.json - Comprehensive model information (JSON)")
print(f"  6. model_metadata.joblib - Comprehensive model information (joblib)")
print(f"\nAll files saved in: {models_dir.absolute()}")

# Print file sizes
print("\n" + "="*80)
print("FILE SIZES")
print("="*80)
total_size = 0
for file_name in ['model.safetensors', 'config.json', 'tokenizer.json', 'vocab.txt', 
                  'label_mapping.joblib', 'model_metadata.json', 'model_metadata.joblib']:
    file_path = models_dir / file_name
    if file_path.exists():
        size_mb = file_path.stat().st_size / (1024 * 1024)
        total_size += size_mb
        print(f"{file_name:<35} {size_mb:>8.2f} MB")

print(f"{'='*35} {total_size:>8.2f} MB")
print(f"\nTotal size of saved model components: {total_size:.2f} MB")


SAVING MODEL AND COMPONENTS
✓ BERT model saved to: saved_models\bert_spam_detector\model.safetensors
✓ Tokenizer saved to: saved_models\bert_spam_detector\tokenizer.json
✓ Label mapping saved to: saved_models\bert_spam_detector\label_mapping.joblib
✓ Model metadata saved to: saved_models\bert_spam_detector\model_metadata.json
✓ Model metadata (joblib) saved to: saved_models\bert_spam_detector\model_metadata.joblib

MODEL COMPONENTS SAVED SUCCESSFULLY

Saved files:
  1. model.safetensors - Trained BERT model weights
  2. config.json - Model configuration
  3. tokenizer.json, vocab.txt - Tokenizer files
  4. label_mapping.joblib - Label mapping (ham=0, spam=1)
  5. model_metadata.json - Comprehensive model information (JSON)
  6. model_metadata.joblib - Comprehensive model information (joblib)

All files saved in: d:\Spam-and-Malware-Detection-AI-model\saved_models\bert_spam_detector

FILE SIZES
model.safetensors                     255.43 MB
config.json                             0.00